# Interacting with CLIP

This is a self-contained notebook that shows how to download and run CLIP models, calculate the similarity between arbitrary image and text inputs, and perform zero-shot image classifications.

# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will install the `clip` package and its dependencies, and check if PyTorch 1.7.1 or later is installed.

In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vugtggus
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-vugtggus
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369499 sha256=c947878e719c68b36b504f4442f9fe593c437030d880727a371ac27650f21031
  Stored in directory: /tmp/pip-ephem-wheel-cache-likvserf/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [3]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"

Torch version: 2.0.1+cu118


# Loading the model

`clip.available_models()` will list the names of available CLIP models.

In [4]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [5]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 46.3MiB/s]


Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


# Image Preprocessing

We resize the input images and center-crop them to conform with the image resolution that the model expects. Before doing so, we will normalize the pixel intensity using the dataset mean and standard deviation.

The second return value from `clip.load()` contains a torchvision `Transform` that performs this preprocessing.



In [6]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x78fb3306a4d0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

# Text Preprocessing

We use a case-insensitive tokenizer, which can be invoked using `clip.tokenize()`. By default, the outputs are padded to become 77 tokens long, which is what the CLIP models expects.

In [7]:
clip.tokenize("Hello World!")

tensor([[49406,  3306,  1002,   256, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)

# CLIP Encoders

In [8]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [9]:
from torch import nn
import copy
from collections import OrderedDict

class LayerNorm(nn.LayerNorm):
    """Subclass torch's LayerNorm to handle fp16."""

    def forward(self, x: torch.Tensor):
        orig_type = x.dtype
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)

class QuickGELU(nn.Module):
    def forward(self, x: torch.Tensor):
        return x * torch.sigmoid(1.702 * x)

In [ ]:
class MyResidualAttentionBlock(nn.Module):
    def __init__(self, d_model: int, n_head: int, attn_mask: torch.Tensor = None):
        super().__init__()

        ### Q1 TODO ###

        self.attn = None
        self.ln_1 = None
        self.mlp = nn.Sequential(OrderedDict([
            ("c_fc", None),
            ("gelu", None),
            ("c_proj", None)
        ]))
        self.ln_2 = None
        self.attn_mask = attn_mask

        ###############

    def attention(self, x: torch.Tensor):
        self.attn_mask = self.attn_mask.to(dtype=x.dtype, device=x.device) if self.attn_mask is not None else None
        return self.attn(x, x, x, need_weights=False, attn_mask=self.attn_mask)[0]

    def forward(self, x: torch.Tensor):

        ### Q1 TODO ###

        x = None
        x = None

        ###############

        return x




In [ ]:
attn_block = copy.deepcopy(model.visual.transformer.resblocks[0]).float().cpu()
my_block = MyResidualAttentionBlock(768, 12)
my_block.load_state_dict(attn_block.state_dict())

N, L, D = 4, 49, 768
x = torch.rand(L, N, D)
a = attn_block(x)
b = my_block(x)
attn_block(x).equal(my_block(x))

# Setting up input images and texts

We are going to feed 8 example images and their textual descriptions to the model, and compare the similarity between the corresponding features.

The tokenizer is case-insensitive, and we can freely give any suitable textual descriptions.

In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline

# images in skimage to use and their textual descriptions
descriptions = {
    "page": "a page of text about segmentation",
    "chelsea": "a facial photo of a tabby cat",
    "astronaut": "a portrait of an astronaut with the American flag",
    "rocket": "a rocket standing on a launchpad",
    "motorcycle_right": "a red motorcycle standing in a garage",
    "camera": "a person looking at a camera on a tripod",
    "horse": "a black-and-white silhouette of a horse",
    "coffee": "a cup of coffee on a saucer"
}

In [ ]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

for filename in [filename for filename in os.listdir(skimage.data_dir) if filename.endswith(".png") or filename.endswith(".jpg")]:
    name = os.path.splitext(filename)[0]
    if name not in descriptions:
        continue

    image = Image.open(os.path.join(skimage.data_dir, filename)).convert("RGB")

    plt.subplot(2, 4, len(images) + 1)
    plt.imshow(image)
    plt.title(f"{filename}\n{descriptions[name]}")
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))
    texts.append(descriptions[name])

plt.tight_layout()


## Building features

We normalize the images, tokenize each text input, and run the forward pass of the model to get the image and text features.

In [ ]:
image_input = torch.tensor(np.stack(images)).cuda()
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).cuda()

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

## Calculating cosine similarity

We normalize the features and calculate the dot product of each pair.

In [ ]:
def get_similarity(image_features, text_features):

    ### Q2 TODO ###




    similarity = None

    ###############

    return similarity

similarity = get_similarity(image_features, text_features)

In [ ]:
count = len(descriptions)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)

## Calculating contrastive loss

We normalize the features and calculate the InfoNCE loss within the 8 image-text pairs.

In [ ]:
import torch.nn.functional as F

def InfoNCE(similarity):
    similarity = torch.from_numpy(similarity)
    n = similarity.shape[0]

    ### Q3 TODO ###



    loss = None

    ###############

    return loss

print(InfoNCE(similarity))
print(InfoNCE(similarity[:4, :4]))

# Zero-Shot Image Classification

You can classify images using the cosine similarity (times 100) as the logits to the softmax operation.

In [ ]:
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=True)
cifar100_test = CIFAR100(os.path.expanduser("~/.cache"), train=False, transform=preprocess, download=True)

prompt = "this is a photo of a {}"
text_descriptions = [prompt.format(label) for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()

In [ ]:
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

In [ ]:
plt.figure(figsize=(16, 16))

for i, image in enumerate(original_images):
    plt.subplot(4, 4, 2 * i + 1)
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(4, 4, 2 * i + 2)
    y = np.arange(top_probs.shape[-1])
    plt.grid()
    plt.barh(y, top_probs[i])
    plt.gca().invert_yaxis()
    plt.gca().set_axisbelow(True)
    plt.yticks(y, [cifar100.classes[index] for index in top_labels[i].numpy()])
    plt.xlabel("probability")

plt.subplots_adjust(wspace=0.5)
plt.show()

## Evaluate zero-shot CLIP on CIFAR 100

In [ ]:
from torch.utils.data import DataLoader
import tqdm

target = cifar100.targets
target_test = cifar100_test.targets

train_loader = DataLoader(cifar100, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(cifar100_test, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
prompt = "a photo of a {}"
text_descriptions = [prompt.format(label) for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()

text_features = model.encode_text(text_tokens).float()
text_features /= text_features.norm(dim=-1, keepdim=True)

In [ ]:
@torch.no_grad()
def evaluate():
    acc = 0.0
    num_samples = len(target_test)

    for batch in tqdm.tqdm(test_loader):
        input_image = batch[0].cuda()
        input_label = batch[1].cuda()

        ### Q4 TODO ###

        image_features = None
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = None

        ###############

        probs = (100.0 * similarity).softmax(dim=-1)

        top_label = torch.argmax(probs, dim=1)
        prediction = torch.eq(input_label, top_label)
        acc += prediction.sum()

    accuracy = acc / num_samples

    return accuracy.item()

print("Acc: , ", evaluate())

## Linear Probing

In [ ]:
class LinearProbing(nn.Module):
    def __init__(self, num_classes=100):
        super(LinearProbing, self).__init__()
        self.fully_connected = nn.Linear(512, num_classes, bias=True)

    def forward(self, x):
        x = self.fully_connected(x)
        return x

lp_model = LinearProbing().cuda()

In [ ]:
def train_one_epoch(train_loader):
    running_loss = 0.0
    last_loss = 0.0
    num_samples = len(cifar100)
    optimizer = torch.optim.SGD(lp_model.parameters(), lr=2e-2, momentum=0.9)

    for i, batch in tqdm.tqdm(enumerate(train_loader)):

        input_image = batch[0].cuda()
        input_label = batch[1]

        labels = torch.eye(100)[input_label].cuda()

        optimizer.zero_grad()

        ### Q5 TODO ###

        image_features = None

        outputs = None

        ###############

        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:
            last_loss = running_loss / 100 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.0

    return last_loss

In [ ]:
@torch.no_grad()
def evaluate_lp():
    acc = 0.0
    num_samples = len(target_test)

    for batch in tqdm.tqdm(test_loader):
        input_image = batch[0].cuda()
        input_label = batch[1].cuda()

        image_features = model.encode_image(input_image).float()
        #image_features /= image_features.norm(dim=-1, keepdim=True)

        outputs = lp_model(image_features)
        probs = outputs.softmax(dim=-1)

        top_label = torch.argmax(probs, dim=1)
        prediction = torch.eq(input_label, top_label)
        acc += prediction.sum()

    accuracy = acc / num_samples

    return accuracy.item()

print("Acc: , ", evaluate_lp())

In [ ]:
epoch_number = 0
EPOCHS = 1

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))
    model.train(False)
    lp_model.train(True)
    avg_loss = train_one_epoch(train_loader)
    print(evaluate_lp())

# Applications: ZegFormer

In [ ]:
# Install detectron2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!git clone https://github.com/dingjiansw101/ZegFormer.git
%cd ZegFormer

import sys
sys.path.insert(0,'/content/ZegFormer')

In [ ]:
!pip install timm
!pip install -r requirements.txt

In [ ]:
import argparse
import glob
import multiprocessing as mp
import os

# fmt: off
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
# fmt: on

import tempfile
import time
import warnings

import cv2
import numpy as np
import tqdm

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.utils.logger import setup_logger

from mask_former import add_mask_former_config
from demo.predictor import VisualizationDemo

In [ ]:
import json
lst = ["sky", "airplane", "road"]
json.dump(lst, open("cls.json", "w"))

In [ ]:
!wget http://images.cocodataset.org/val2017/000000005477.jpg -q -O input.jpg
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1bA6DXr9VOMsRkU0vyY2EpGRkyQnhnze3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1bA6DXr9VOMsRkU0vyY2EpGRkyQnhnze3" -O weights.pth && rm -rf ~/cookies.txt

In [ ]:
import atexit
import bisect
import multiprocessing as mp
from collections import deque

import cv2
import torch

from detectron2.data import MetadataCatalog
from detectron2.engine.defaults import DefaultPredictor
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from types import SimpleNamespace


class VisualizationDemo(object):
    def __init__(self, cfg, instance_mode=ColorMode.IMAGE, parallel=False):
        """
        Args:
            cfg (CfgNode):
            instance_mode (ColorMode):
            parallel (bool): whether to run the model in different processes from visualization.
                Useful since the visualization logic can be slow.
        """
        self.metadata = MetadataCatalog.get(
            cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused"
        )
        ns = SimpleNamespace()
        ns.stuff_classes = demo.predictor.model.sem_seg_head.predictor.test_class_texts
        self.metadata = None
        self.cpu_device = torch.device("cpu")
        self.instance_mode = instance_mode

        self.predictor = DefaultPredictor(cfg)

        ns.stuff_classes = self.predictor.model.sem_seg_head.predictor.test_class_texts
        self.metadata = ns

    def run_on_image(self, image):
        """
        Args:
            image (np.ndarray): an image of shape (H, W, C) (in BGR order).
                This is the format used by OpenCV.
        Returns:
            predictions (dict): the output of the model.
            vis_output (VisImage): the visualized image output.
        """
        vis_output = None
        predictions = self.predictor(image)
        # Convert image from OpenCV BGR format to Matplotlib RGB format.
        image = image[:, :, ::-1]
        visualizer = Visualizer(image, self.metadata, instance_mode=self.instance_mode)
        vis_output = visualizer.draw_sem_seg(
                predictions["sem_seg"].argmax(dim=0).to(self.cpu_device)
            )


        return predictions, vis_output

In [ ]:
import torch
import easydict
def setup_cfg(args):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_mask_former_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)

    cfg.MODEL.SEM_SEG_HEAD.TRAIN_CLASS_JSON = "cls.json"
    cfg.MODEL.SEM_SEG_HEAD.TEST_CLASS_JSON = "cls.json"

    cfg.freeze()
    return cfg

def get_parser():
    parser = argparse.ArgumentParser(description="Detectron2 demo for builtin configs")
    parser.add_argument(
        "--config-file",
        default="configs/ade20k-150/maskformer_R50_bs16_160k.yaml",
        metavar="FILE",
        help="path to config file",
    )
    parser.add_argument(
        "--input",
        nargs="+",
        default="input.png",
        help="A list of space separated input images; "
        "or a single glob pattern such as 'directory/*.jpg'",
    )
    parser.add_argument(
        "--output",
        default="./",
        help="A file or directory to save output visualizations. "
        "If not given, will show output in an OpenCV window.",
    )

    parser.add_argument(
        "--confidence-threshold",
        type=float,
        default=0.5,
        help="Minimum score for instance predictions to be shown",
    )
    parser.add_argument(
        "--opts",
        help="Modify config options using the command-line 'KEY VALUE' pairs",
        default=[],
        nargs=argparse.REMAINDER,
    )
    return parser

args = easydict.EasyDict({"config_file": "configs/coco-stuff/zegformer_R101_bs32_60k_vit16_coco-stuff_gzss_eval.yaml",
                          "input": "input.png",
                          "output": "./",
                          "opts": ["MODEL.WEIGHTS", "weights.pth", "MODEL.MASK_FORMER.ENSEMBLING_ALL_CLS", "False", "MODEL.MASK_FORMER.ENSEMBLING", "False"]})
cfg = setup_cfg(args)
demo = VisualizationDemo(cfg)
demo.predictor.model.gzero_calibrate = -1.0

In [ ]:
img = read_image("input.jpg", format="BGR")
predictions, visualized_output = demo.run_on_image(img)
visualized_output.fig